In [3]:
%matplotlib inline
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.fftpack import dct
import matplotlib.pyplot as plt
from python_speech_features.base import mfcc
import scipy.io.wavfile
from seq2seq.models.attention_seq2seq import AttentionSeq2Seq

In [9]:
root = './'

In [10]:
class FileOpen(tf.gfile.Open):
    def seek(self, position, whence = 0):
        if (whence == 0):
            tf.gfile.Open.seek(self, position)
        elif (whence == 1):
            tf.gfile.Open.seek(self, self.tell() + position)
        else:
            raise FileError

In [11]:
chars = np.asarray(list(" '+-.ABCDEFGHIJKLMNOPQRSTUVWXYZ_"))

def onehot(x):
    x = np.asarray(x)
    return np.tile(x, (32, 1)).T == chars # 32 = different chars

In [7]:
session = tf.InteractiveSession()

with session.as_default():
    model = AttentionSeq2Seq(AttentionSeq2Seq.default_params(), tf.contrib.learn.ModeKeys.TRAIN)

INFO:tensorflow:Creating AttentionSeq2Seq in mode=train
INFO:tensorflow:
AttentionSeq2Seq:
  attention.class: AttentionLayerBahdanau
  attention.params: {}
  bridge.class: seq2seq.models.bridges.ZeroBridge
  bridge.params: {}
  decoder.class: seq2seq.decoders.AttentionDecoder
  decoder.params: {}
  embedding.dim: 100
  embedding.init_scale: 0.04
  embedding.share: false
  encoder.class: seq2seq.encoders.BidirectionalRNNEncoder
  encoder.params: {}
  inference.beam_search.beam_width: 0
  inference.beam_search.choose_successors_fn: choose_top_k
  inference.beam_search.length_penalty_weight: 0.0
  optimizer.clip_embed_gradients: 0.1
  optimizer.clip_gradients: 5.0
  optimizer.learning_rate: 0.0001
  optimizer.lr_decay_rate: 0.99
  optimizer.lr_decay_steps: 100
  optimizer.lr_decay_type: ''
  optimizer.lr_min_learning_rate: 1.0e-12
  optimizer.lr_staircase: false
  optimizer.lr_start_decay_at: 0
  optimizer.lr_stop_decay_at: 2147483647
  optimizer.name: Adam
  optimizer.params: {}
  optimi

In [12]:
for trans in tf.gfile.Open(root + 'wsj0/transcripts/wsj0/wsj0.trans').readlines():
    cont, file = trans.split('(')
    file = file[:-2]
    sample_rate, signal = scipy.io.wavfile.read(FileOpen(root + file.rstrip('\n'), 'rb'))
    X = onehot(list(cont))
    Y = mfcc(signal, sample_rate)
    model(X, Y, AttentionSeq2Seq.default_params())

NotFoundError: ./11_10_1/wsj0/sd_tr_s/00f/00fa0101.wv2